In [2]:
import os 
import pandas as pd 
import numpy as np 
import re 

In [3]:
# path to folders having .md files
folder = "../papers/"

In [4]:
file = "../papers/39834406.md"

In [34]:
import os
import json

def markdown_parser(folder_path):
    articles = []

    for filename in os.listdir(folder_path):
        if filename.endswith(".md"):
            with open(os.path.join(folder_path, filename), 'r', encoding='utf-8') as f:
                lines = f.readlines()

            article = {
                "pubmed_id": None,
                "title": None,
                "journal": None,
                "keywords": None,
                "abstract": None,
#                 "full_text": "".join(lines)
            }

            # Extract title
            if lines and lines[0].startswith("# "):
                article["title"] = lines[0][2:].strip()

            # Extract fields using line-by-line parsing
            in_abstract = False
            abstract_lines = []
            for line in lines:
                stripped = line.strip()

                if stripped.startswith("**PMID:**"):
                    article["pubmed_id"] = stripped.replace("**PMID:**", "").strip()

                elif stripped.startswith("**Journal:**"):
                    article["journal"] = stripped.replace("**Journal:**", "").strip()

                elif stripped.startswith("**Keywords:**"):
                    article["keywords"] = stripped.replace("**Keywords:**", "").strip()

                elif stripped.startswith("## Abstract"):
                    in_abstract = True
                    continue

                elif in_abstract and stripped.startswith("##"):
                    # End of abstract section
                    in_abstract = False

                elif in_abstract:
                    abstract_lines.append(stripped)

            if abstract_lines:
                article["abstract"] = " ".join(abstract_lines)

            articles.append(article)

    return articles


In [35]:
papers = markdown_parser(folder)

In [36]:
len(papers)

50

In [49]:
with open("../outputs/all_papers.json", "w", encoding="utf-8") as f:
    json.dump(papers, f, indent=2)

In [52]:
df = pd.DataFrame(papers)

In [54]:
df.to_csv("../outputs/parsed_md_files.csv", index = False)

In [55]:
df.shape

(50, 5)

In [56]:
df.columns

Index(['pubmed_id', 'title', 'journal', 'keywords', 'abstract'], dtype='object')

In [57]:
df.sample(3)

,pubmed_id,title,journal,keywords,abstract
10,40230848,The role of transketolase in the immunotherapy...,Frontiers in immunology (Front Immunol),"Hep-G2, TKT, hepatocellular carcinoma, pentose...",To explore the role of transketolase (TKT) in...
29,40166174,High-throughput single cell -omics using semi-...,bioRxiv : the preprint server for biology (bio...,None,Biological systems are inherently complex and...
27,40241135,ADAR1-high tumor-associated macrophages induce...,Molecular cancer (Mol Cancer),"Biomarker, Chemoresistance, Colorectal cancer,...",Colorectal cancer (CRC) is considered the thi...
